In [1]:
import os
import json
import  pandas as pd
import traceback # type: ignore
print("hello")

hello


In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
key_openai = os.getenv("OPENAI_API_KEY")

In [13]:
llm = ChatOpenAI(openai_api_key=key_openai,model_name="gpt-3.5-turbo",temperature=0.7)

c:\Users\dell\Desktop\GEN AI Proj\MCQ\GenAI\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [14]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=key_openai,model_name="gpt-3.5-turbo",temperature=0.7)

In [15]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001924D488DF0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001924D48A290>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [17]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [18]:
RESPONSE_JSON ={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },
    "4":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer"
    },

}

In [19]:
TEMPLATE = """
Text:{text}
You are an export MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check the all questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as guide \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [21]:
quize_prompt_template = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
    
)

In [23]:
quize_chain = LLMChain(llm=llm,prompt=quize_prompt_template,output_key="quiz",verbose=True)

In [24]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [26]:
quiz_evlution_prompt = PromptTemplate(input_variables=['subject',"quiz"],template=TEMPLATE2)

In [30]:
review_chian = LLMChain(llm=llm,prompt=quiz_evlution_prompt,output_key="review",verbose=True)

In [31]:
generate_evaluate_chain = SequentialChain(chains=[quize_chain,review_chian],input_variables=["text","number","subject","tone","response_json",'subject'],
                output_variables=["quiz","review"],verbose=True)

In [33]:
file_path = r"C:\Users\dell\Desktop\GEN AI Proj\MCQ\GenAI\data.txt"
with open(file_path,'r') as file:
    TEXT= file.read()

In [35]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [36]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="simple"

In [38]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [40]:
print(f"Total Tokens : {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens : 1870
Prompt Tokens: 1437
Completion Tokens: 433
Total Cost: 0.0030215


In [42]:
print(response.get("quiz"))


{
"1": {
"mcq": "What term was coined by Arthur Samuel in 1959 to describe the field of computer gaming and artificial intelligence?",
"options": {
"a": "Machine Thinking",
"b": "Artificial Learning",
"c": "Machine Learning",
"d": "Computer Intelligence"
},
"correct": "c"
},
"2": {
"mcq": "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
"options": {
"a": "Tom M. Mitchell",
"b": "Donald Hebb",
"c": "Walter Pitts",
"d": "Warren McCulloch"
},
"correct": "b"
},
"3": {
"mcq": "In the 1960s, what was the name of the 'learning machine' developed by Raytheon Company to analyze sonar signals and speech patterns?",
"options": {
"a": "Cyberdyne",
"b": "Cybernet",
"c": "Cybertron",
"d": "Cyberspace"
},
"correct": "c"
},
"4": {
"mcq": "According to Tom M. Mitchell, what is the definition of a computer program learning from experience with respect to tasks and performance?",
"options": {
"a": "Improving with experience E",
"b": "Learning 

In [43]:
quiz = json.loads(response.get("quiz"))
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [44]:
quiz_table_data

[{'MCQ': 'What term was coined by Arthur Samuel in 1959 to describe the field of computer gaming and artificial intelligence?',
  'Choices': 'a: Machine Thinking | b: Artificial Learning | c: Machine Learning | d: Computer Intelligence',
  'Correct': 'c'},
 {'MCQ': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure?",
  'Choices': 'a: Tom M. Mitchell | b: Donald Hebb | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': "In the 1960s, what was the name of the 'learning machine' developed by Raytheon Company to analyze sonar signals and speech patterns?",
  'Choices': 'a: Cyberdyne | b: Cybernet | c: Cybertron | d: Cyberspace',
  'Correct': 'c'},
 {'MCQ': 'According to Tom M. Mitchell, what is the definition of a computer program learning from experience with respect to tasks and performance?',
  'Choices': 'a: Improving with experience E | b: Learning from experience T | c: Performing with measure P | d: Defining w

In [46]:
quiz_df = pd.DataFrame(quiz_table_data)

In [47]:
quiz_df

,MCQ,Choices,Correct
0,What term was coined by Arthur Samuel in 1959 ...,a: Machine Thinking | b: Artificial Learning |...,c
1,Who published the book 'The Organization of Be...,a: Tom M. Mitchell | b: Donald Hebb | c: Walte...,b
2,"In the 1960s, what was the name of the 'learni...",a: Cyberdyne | b: Cybernet | c: Cybertron | d:...,c
3,"According to Tom M. Mitchell, what is the defi...",a: Improving with experience E | b: Learning f...,a
4,What are the two main objectives of modern-day...,a: To write code and analyze data | b: To pred...,b


In [49]:
quiz_df.to_csv("machine_learning_quiz.csv")

In [50]:
from langchain_community.callbacks import get_openai_callback